# I. Pen-and-paper [11v]
#### version 2 changes in blue (21/10/23)
### Given the following observations, {(1 0.6 0.1) , (0 −0.4 0.8) , (0 0.2 0.5) , (1 0.4 −0.1)}. Consider a Bayesian clustering that assumes independence {𝑦1} and {𝑦2, 𝑦3}, two clusters following a Bernoulli distribution on 𝑦1 (𝑝1 and 𝑝2), a multivariate Gaussian on {𝑦2, 𝑦3} (𝑁1 and 𝑁2), and the following initial mixture:
### <center> 𝜋1 = 0.5, 𝜋2 = 0.5 </center>
### <center> 𝑝1 = 𝑃(𝑦1 = 1) = 0.3,  𝑝2 = 𝑃(𝑦1 = 1) = 0.7  </center>
### <center> 𝑁1 (𝐮1 = (1 1) , 𝚺1 = (2 0.5, 0.5 2)), 𝑁2 (𝐮2 = (0 0) , 𝚺2 = (1.5 1, 1 1.5)).  </center>
### Note: you can solve this exercise by neglecting 𝑦1 and still scoring up to 70% of its grade.

### 1) [6v] Perform one epoch of the EM clustering algorithm and determine the new parameters. <br>Hint: we suggest you to use numpy and scipy, however disclose the intermediary results step by step.

In [1]:
import numpy as np

# priors
pi = np.array([0.5, 0.5])

# Cluster 1 (N1)
mu1 = np.array([1, 1])
cov1 = np.array([[2, 0.5], [0.5, 2]])

# Cluster 2 (N2)
mu2 = np.array([0, 0])
cov2 = np.array([[1.5, 1], [1, 1.5]])

p1 = 0.3
p2 = 0.7

In [2]:
from scipy.stats import multivariate_normal


# Given observations
observations = np.array([
    [1, 0.6, 0.1],
    [0, -0.4, 0.8],
    [0, 0.2, 0.5],
    [1, 0.4, -0.1]
])
observations_y2_y3 = observations[:, 1:]  # Considering only y2 and y3

# Number of observations
num_observations = len(observations)

# Initializing responsibilities
responsibilities = np.zeros((num_observations, 2))

for i in range(num_observations):
    # Calculate the probability of each observation (y2, y3) belonging to each cluster
    prob_cluster1 = pi[0] * multivariate_normal.pdf(observations_y2_y3[i], mean=mu1, cov=cov1)
    print("Gaussian 1, observation ", i+1, ":", multivariate_normal.pdf(observations_y2_y3[i], mean=mu1, cov=cov1))
    prob_cluster2 = pi[1] * multivariate_normal.pdf(observations_y2_y3[i], mean=mu2, cov=cov2)
    print("Gaussian 2, observation ", i+1, ":", multivariate_normal.pdf(observations_y2_y3[i], mean=mu2, cov=cov2))
    
    # Multiply by y1 probabilities for each cluster
    prob_cluster1 *= p1 if observations[i, 0] == 1 else (1 - p1)
    prob_cluster2 *= p2 if observations[i, 0] == 1 else (1 - p2)
    
    print("probs before normalization", [prob_cluster1, prob_cluster2])
    # Normalize the responsibilities
    responsibilities[i] = [prob_cluster1 / (prob_cluster1 + prob_cluster2),
                           prob_cluster2 / (prob_cluster1 + prob_cluster2)]
    
    print(responsibilities[i])


Gaussian 1, observation  1 : 0.06657529920303393
Gaussian 2, observation  1 : 0.11961837142058572
probs before normalization [0.009986294880455089, 0.041866429997205]
[0.19258959 0.80741041]
Gaussian 1, observation  2 : 0.05004888824270901
Gaussian 2, observation  2 : 0.0681905803254947
probs before normalization [0.017517110884948152, 0.010228587048824206]
[0.63134512 0.36865488]
Gaussian 1, observation  3 : 0.06837452355368487
Gaussian 2, observation  3 : 0.12958103481626038
probs before normalization [0.023931083243789703, 0.019437155222439058]
[0.55181128 0.44818872]
Gaussian 1, observation  4 : 0.059046993443730274
Gaussian 2, observation  4 : 0.12450008976589248
probs before normalization [0.00885704901655954, 0.04357503141806237]
[0.16892423 0.83107577]


In [17]:
# Update the parameters
# Calculate new pi (cluster weights)
new_pi = np.mean(responsibilities, axis=0)

# Calculate new means for clusters (considering y2 and y3 only)
new_mu1 = np.dot(responsibilities[:, 0], observations[:, 1:]) / np.sum(responsibilities[:, 0])
new_mu2 = np.dot(responsibilities[:, 1], observations[:, 1:]) / np.sum(responsibilities[:, 1])

# Calculate new covariance matrices for clusters (considering y2 and y3 only)
diff1 = observations[:, 1:] - new_mu1
cov1 = np.dot(responsibilities[:, 0] * diff1.T, diff1) / np.sum(responsibilities[:, 0])

diff2 = observations[:, 1:] - new_mu2
cov2 = np.dot(responsibilities[:, 1] * diff2.T, diff2) / np.sum(responsibilities[:, 1])

# Update the Bernoulli probabilities
p1 = np.sum(responsibilities[:, 0] * observations[:, 0]) / np.sum(responsibilities[:, 0])
p2 = np.sum(responsibilities[:, 1] * observations[:, 0]) / np.sum(responsibilities[:, 1])

# Update the parameters
pi = new_pi
mu1 = new_mu1
mu2 = new_mu2
cov1 = cov1
cov2 = cov2
p1 = p1
p2 = p2


In [18]:
print("Priors:", pi)
print("Means u1:", mu1)
print("Sigma 1:", cov1)
print("Means u2:", mu2)
print("Sigma 2:", cov2)
print("New P1:", p1)
print("New P2:", p2)

Priors: [0.38616755 0.61383245]
Means u1: [0.026509   0.50712978]
Sigma 1: [[ 0.14136501 -0.10540546]
 [-0.10540546  0.0960526 ]]
Means u2: [0.30914476 0.2104205 ]
Sigma 2: [[ 0.10829305 -0.08865175]
 [-0.08865175  0.1041233 ]]
New P1: 0.23403948408541084
New P2: 0.6673181710330361


### 2) [2v] Given the new observation, X𝑛𝑒𝑤 = (1 0.3 0.7), determine the cluster memberships (posteriors).

In [15]:
# New observation
x_new = np.array([1, 0.3, 0.7])

# Calculate probabilities of the new observation (y2, y3) belonging to each cluster
prob_cluster1 = pi[0] * multivariate_normal.pdf(x_new[1:], mean=mu1, cov=cov1)
print("Gaussian 1:", multivariate_normal.pdf(x_new[1:], mean=mu1, cov=cov1))
prob_cluster2 = pi[1] * multivariate_normal.pdf(x_new[1:], mean=mu2, cov=cov2)
print("Gaussian 2:", multivariate_normal.pdf(x_new[1:], mean=mu2, cov=cov2))

# Multiply by y1 probabilities for each cluster
prob_cluster1 *= p1 if x_new[0] == 1 else (1 - p1)
prob_cluster2 *= p2 if x_new[0] == 1 else (1 - p2)

print("Posterior 1 before normalization:", prob_cluster1)
print("Posterior 2 before normalization:", prob_cluster2)
# Calculate the posteriors (responsibilities) for the new observation
posterior_cluster1 = prob_cluster1 / (prob_cluster1 + prob_cluster2)
posterior_cluster2 = prob_cluster2 / (prob_cluster1 + prob_cluster2)

print("Posterior for Cluster 1:", posterior_cluster1)
print("Posterior for Cluster 2:", posterior_cluster2)


Gaussian 1: 0.027075573673303183
Gaussian 2: 0.06843088109574726
Posterior 1 before normalization: 0.002447048524076359
Posterior 2 before normalization: 0.028030763221841403
Posterior for Cluster 1: 0.08028950846197545
Posterior for Cluster 2: 0.9197104915380245


### 3) [2.5v] Performing a hard assignment of observations to clusters under a ML assumption, identify the silhouette of the larger cluster under a Manhattan distance.

In [35]:
assigned_clusters = []  # List to hold assigned clusters

for i in range(num_observations):
    # Calculate the probability of each observation (y2, y3) belonging to each cluster
    prob_cluster1 = multivariate_normal.pdf(observations_y2_y3[i], mean=mu1, cov=cov1)
    #print("Gaussian 1, observation ", i+1, ":", multivariate_normal.pdf(observations_y2_y3[i], mean=mu1, cov=cov1))
    prob_cluster2 = multivariate_normal.pdf(observations_y2_y3[i], mean=mu2, cov=cov2)
    #print("Gaussian 2, observation ", i+1, ":", multivariate_normal.pdf(observations_y2_y3[i], mean=mu2, cov=cov2))
    
    # Multiply by y1 probabilities for each cluster
    prob_cluster1 *= p1 if observations[i, 0] == 1 else (1 - p1)
    prob_cluster2 *= p2 if observations[i, 0] == 1 else (1 - p2)
    
    print("probs under ML assumption", [prob_cluster1, prob_cluster2])    
    
    # Decide the assigned cluster for the observation based on probabilities
    assigned_cluster = 1 if prob_cluster2 > prob_cluster1 else 0
    assigned_clusters.append(assigned_cluster)
    print(f"Observation {i+1} assigned to Cluster {assigned_cluster + 1}\n")

# Print assigned clusters
print("Assigned Clusters:", assigned_clusters)

probs under ML assumption [0.2314743376774436, 0.9495425230090515]
Observation 1 assigned to Cluster 2

probs under ML assumption [1.266332483325112, 0.08873672123767436]
Observation 2 assigned to Cluster 1

probs under ML assumption [1.438110403645913, 0.45417449728697806]
Observation 3 assigned to Cluster 1

probs under ML assumption [0.02076522613490563, 0.7233119821229737]
Observation 4 assigned to Cluster 2

Assigned Clusters: [1, 0, 0, 1]


In [54]:
import numpy as np
from sklearn.metrics import pairwise_distances

# Assuming 'assigned_clusters' and 'observations' are defined in your code

# Calculate pairwise Manhattan distances between observations
manhattan_distances = pairwise_distances(observations, metric='manhattan')

silhouette_scores = []

for i, observation in enumerate(observations):
    cluster_index = assigned_clusters[i]
    if cluster_index == 1:
        other_cluster_index = 0  # Index of the other cluster
    else:
        other_cluster_index = 1

    # Calculate the average distance to other points in the same cluster
    same_cluster_mask = np.array(assigned_clusters) == cluster_index
    same_cluster_mask[i] = False  # Exclude the current index 'i'
    same_cluster_distances = manhattan_distances[i][same_cluster_mask]
    a = np.mean(same_cluster_distances)

    # Calculate the average distance to points in the nearest cluster
    nearest_cluster_mask = np.array(assigned_clusters) == other_cluster_index
    nearest_cluster_mask[i] = False  # Exclude the current index 'i'
    nearest_cluster_distances = manhattan_distances[i][nearest_cluster_mask]
    b = np.mean(nearest_cluster_distances)

    # Calculate silhouette score for the observation
    silhouette = (b - a) / np.maximum(a, b)
    silhouette_scores.append(silhouette)

# Print silhouette scores for each observation
for i, score in enumerate(silhouette_scores):
    print(f"Observation {i + 1}: Silhouette Score -> {score}")

silhouette_cluster_1 = [silhouette_scores[i] for i in range(len(assigned_clusters)) if assigned_clusters[i] == 0]
print("Cluster C1 Silhouette Score:", sum(silhouette_cluster_1)/len(silhouette_cluster_1) )
silhouette_cluster_2 = [silhouette_scores[i] for i in range(len(assigned_clusters)) if assigned_clusters[i] == 1]
print("Cluster C2 Silhouette Score:", sum(silhouette_cluster_2)/len(silhouette_cluster_2) )

Observation 1: Silhouette Score -> 0.8222222222222223
Observation 2: Silhouette Score -> 0.6666666666666666
Observation 3: Silhouette Score -> 0.4999999999999999
Observation 4: Silhouette Score -> 0.8222222222222223
Cluster C1 Silhouette Score: 0.5833333333333333
Cluster C2 Silhouette Score: 0.8222222222222223


In [41]:
from scipy.spatial import distance

silhouette_scores = []

for i, observation in enumerate(observations):
    cluster_index = assigned_clusters[i]
    if cluster_index == 1:
        other_cluster_index = 0  # Assign the opposite cluster index
    else:
        other_cluster_index = 1

    # Calculate the average distance to other points in the same cluster using the a_fn function
    same_cluster_distances = [
        distance.cityblock(observation, obs) for obs, idx in zip(observations, assigned_clusters) if idx == cluster_index and not np.array_equal(observation, obs)
    ]
    a = sum(same_cluster_distances) / len(same_cluster_distances)

    # Calculate the average distance to points in the nearest cluster using the a_fn function
    nearest_cluster_distances = [
        distance.cityblock(observation, obs) for obs, idx in zip(observations, assigned_clusters) if idx == other_cluster_index
    ]
    b = sum(nearest_cluster_distances) / len(nearest_cluster_distances)

    # Calculate silhouette score for the observation
    silhouette = (b - a) / max(a, b)
    silhouette_scores.append(silhouette)

# Print silhouette scores for each observation
for i, score in enumerate(silhouette_scores):
    print(f"Observation {i + 1}: Silhouette Score -> {score}")

silhouette_cluster_1 = [silhouette_scores[i] for i in range(len(assigned_clusters)) if assigned_clusters[i] == 0]
print("Cluster C1 Silhouette Score:", sum(silhouette_cluster_1)/len(silhouette_cluster_1) )
silhouette_cluster_2 = [silhouette_scores[i] for i in range(len(assigned_clusters)) if assigned_clusters[i] == 1]
print("Cluster C2 Silhouette Score:", sum(silhouette_cluster_2)/len(silhouette_cluster_2) )

Observation 1: Silhouette Score -> 0.8222222222222223
Observation 2: Silhouette Score -> 0.6666666666666666
Observation 3: Silhouette Score -> 0.4999999999999999
Observation 4: Silhouette Score -> 0.8222222222222223
Cluster C1 Silhouette Score: 0.5833333333333333
Cluster C2 Silhouette Score: 0.8222222222222223


### 4) [0.5v] Knowing the purity of the clustering solution is 0.75, identify the number of possible classes (ground truth).

Feita em papel (não necessita de grandes cálculos).